# begin


In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, LSTM, Activation, GRU, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
import keras.callbacks as kcallbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report



#20 normal + 20 anomalous = test / con lai la train
#fwap + http = train

# Path

## Normal

In [ ]:
DATASET_TRAINING = '/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/train.csv'
DATASET_TESTING_csic='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_csic2010.csv'
DATASET_TESTING_fwaf='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_fwaf.csv'
DATASET_TESTING_httpParams='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_httpParams.csv'
DATASET_TESTING_ecml='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_ECML.csv'

## minmax

In [ ]:
# DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/train_minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_csic2010-minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_fwaf-minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_http_param-minmax.csv'

## binary

In [ ]:
# DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/train_minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_csic2010-minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_fwaf-minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_http_param-minmax.csv'

# load data

In [ ]:
### Load data
df_train = pd.read_csv(DATASET_TRAINING)
# display(df_train)
# df_test = pd.read_csv(DATASET_TESTING, header=None, index_col=False)
df_train = df_train.astype(float)
df_train = df_train.sample(frac = 1)
x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, [-1]]
x_train = x_train/255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

y_train = y_train.astype(int)
y_val = y_val.astype(int)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

input_shape = x_train.shape[1]
output_shape = y_train.shape[1]

## load test

In [ ]:
df_test_csic = pd.read_csv(DATASET_TESTING_csic)

df_test_csic = df_test_csic.astype(float)

df_test_csic = df_test_csic.sample(frac = 1)

x_test_csic, y_test_csic = df_test_csic.iloc[:, :-1], df_test_csic.iloc[:, [-1]]

x_test_csic = x_test_csic/255.0
y_test_csic = y_test_csic.astype(int)
y_test_csic = to_categorical(y_test_csic)

In [ ]:
df_test_fwaf = pd.read_csv(DATASET_TESTING_fwaf)

df_test_fwaf = df_test_fwaf.astype(float)

df_test_fwaf = df_test_fwaf.sample(frac = 1)

x_test_fwaf, y_test_fwaf = df_test_fwaf.iloc[:, :-1], df_test_fwaf.iloc[:, [-1]]

x_test_fwaf = x_test_fwaf/255.0
y_test_fwaf = y_test_fwaf.astype(int)
y_test_fwaf = to_categorical(y_test_fwaf)

In [ ]:
df_test_httpParasm = pd.read_csv(DATASET_TESTING_httpParams)

df_test_httpParasm = df_test_httpParasm.astype(float)

df_test_httpParasm = df_test_httpParasm.sample(frac = 1)

x_test_httpParasm, y_test_httpParasm = df_test_httpParasm.iloc[:, :-1], df_test_httpParasm.iloc[:, [-1]]

x_test_httpParasm = x_test_httpParasm/255.0
y_test_httpParasm = y_test_httpParasm.astype(int)
y_test_httpParasm = to_categorical(y_test_httpParasm)

In [ ]:
df_test_ecml = pd.read_csv(DATASET_TESTING_ecml)

df_test_ecml = df_test_ecml.astype(float)

df_test_ecml = df_test_ecml.sample(frac = 1)

x_test_ecml, y_test_ecml = df_test_ecml.iloc[:, :-1], df_test_ecml.iloc[:, [-1]]

x_test_ecml = x_test_ecml/255.0
y_test_ecml = y_test_ecml.astype(int)
y_test_ecml = to_categorical(y_test_ecml)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/UIT/HocKy8/KLTN/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = kcallbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')

# ]

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# CNN


## CNN 1

In [ ]:
# # ### CNN 1
# model = Sequential()
# model.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn

# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# # train
# early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
# # history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, ,train, y_train, epochs=100, batch_size=64, , shuffle=True)

# # # loss: 0.2412 - accuracy: 0.8811

## CNN 2

In [ ]:
### CNN 2
model = Sequential()

model.add(Conv1D(64, 3, activation="relu", padding='same', input_shape=(input_shape, 1)))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation="softmax"))

# define optimizer and objective, compile cnn

#### Chú ý đoạn này
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy',f1_m,precision_m, recall_m]) # for metrics
#### Chú ý đoạn này

early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
#fit model
history=model.fit(x_train, y_train, epochs=20, batch_size=64,  shuffle=True) #epochs=20,

# # loss: 0.2433 - accuracy: 0.8846 - 128
# # loss: 0.2384 - accuracy: 0.8884 - 64
# # loss: 0.2347 - accuracy: 0.8934 - 32

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_csic, y_test_csic, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_fwaf, y_test_fwaf, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_httpParasm, y_test_httpParasm, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_ecml, y_test_ecml, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

## CNN 3

In [ ]:
# ### CNN 3
# model = Sequential()
# model.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

# # loss: 0.2384 - accuracy: 0.8908

## CNN5

In [ ]:
# ### CNN5
# model = Sequential()
# model.add(Conv1D(16, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

## CNN6

In [ ]:
# ### CNN6
# model = Sequential()
# model.add(Conv1D(16, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
# model.add(Conv1D(64, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(128, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(256, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Conv1D(256, 3, padding="same", activation="relu"))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Flatten())
# model.add(Dense(256, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(output_shape, activation="softmax"))

# # define optimizer and objective, compile cnn
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

## CNN + LSTM

In [ ]:
# ### LSTM
# model = Sequential()
# model.add(LSTM(2, input_shape=(input_shape, 1), return_sequences=True, activation='sigmoid'))
# model.add(LSTM(units=120, recurrent_activation='hard_sigmoid', activation='sigmoid'))
# model.add(Dense(activation='sigmoid', units=output_shape))
# adam=Adam(learning_rate=0.01)

# model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

# # loss: 0.4802 - accuracy: 0.7314

# LSTM

## LSTM 2

In [ ]:
## LSTM 2
model = Sequential()
model.add(LSTM(4,input_shape=(input_shape, 1)))
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy',f1_m,precision_m, recall_m])

history=model.fit(x_train, y_train, epochs=100, batch_size=64, shuffle=True)

# loss: 0.4497 - accuracy: 0.7483

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_csic, y_test_csic, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_fwaf, y_test_fwaf, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_httpParasm, y_test_httpParasm, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

## LSTM 3

In [ ]:
# ### LSTM 3
# model = Sequential()
# model.add(LSTM(8,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(8,input_dim=input_shape, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

## LSTM 4

In [ ]:
# ### LSTM 4
# model = Sequential()
# model.add(LSTM(16,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

## LSTM 5

In [ ]:
# ### LSTM 5
# model = Sequential()
# model.add(LSTM(64,input_dim=input_shape, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(LSTM(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=64, , shuffle=True)

# GRU

## GRU 1

In [ ]:
# ### GRU 1
# model = Sequential()
# model.add(GRU(4,input_shape=(input_shape, 1)))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=32, , shuffle=True)

# # loss: 0.6380 - accuracy: 0.6299

## GRU 2

In [ ]:
### GRU 2
model = Sequential()
model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy',f1_m,precision_m, recall_m])

history=model.fit(x_train, y_train, epochs=30, batch_size=64,  shuffle=True)
# loss: 0.4454 - accuracy: 0.7774

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_csic, y_test_csic, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_fwaf, y_test_fwaf, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_httpParasm, y_test_httpParasm, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

## GRU 3


In [ ]:
# ### GRU 3
# model = Sequential()
# model.add(GRU(16,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, , shuffle=True)
# # loss: 0.3558 - accuracy: 0.8156

## GRU 4

In [ ]:
# ### GRU 4
# model = Sequential()
# model.add(GRU(64,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(GRU(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, , shuffle=True)
# # loss: 0.2769 - accuracy: 0.8616

# SimpleRNN

## SimpleRNN 1

In [ ]:
# ### SimpleRNN 1
# model = Sequential()
# model.add(SimpleRNN(4,input_shape=(input_shape, 1)))
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=100, batch_size=32, , shuffle=True)

# # loss: 0.5145 - accuracy: 0.7150

## SimpleRNN 2

In [ ]:
### SimpleRNN 2
model = Sequential()
model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy',f1_m,precision_m, recall_m])

history=model.fit(x_train, y_train,  epochs=30, batch_size=64,  shuffle=True)
# loss: 0.4497 - accuracy: 0.7519

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_csic, y_test_csic, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_fwaf, y_test_fwaf, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_httpParasm, y_test_httpParasm, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

## SimpleRNN 3

In [ ]:
# ### SimpleRNN 3
# model = Sequential()
# model.add(SimpleRNN(16,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(16, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(16, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, , shuffle=True)
# #loss: 0.3500 - accuracy: 0.8229

## Simple RNN 4

In [ ]:
# ### Simple RNN 4
# model = Sequential()
# model.add(SimpleRNN(64,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64,return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64, return_sequences=True))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64, return_sequences=False))  # try using a GRU instead, for fun
# model.add(Dropout(0.1))
# model.add(Dense(output_shape))
# model.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# history=model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=30, batch_size=64, , shuffle=True)
# # loss: 0.6376 - accuracy: 0.6803

# DNN

In [ ]:
### DNN
model = Sequential()
model.add(Dense(1024,input_dim=input_shape,activation='relu'))
# model.add(Dense(64,input_dim=input_shape,activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy',f1_m,precision_m, recall_m])

history=model.fit(x_train, y_train,  epochs=10, batch_size=64,  shuffle=True) #epochs=100, batch_size=32

# loss: 0.2518 - accuracy: 0.8580

# Evaluate on test data

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_csic, y_test_csic, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_fwaf, y_test_fwaf, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score

In [ ]:

print("Evaluate on test data")
results = model.evaluate(x_test_httpParasm, y_test_httpParasm, batch_size=32)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]
f1_score = 2 * (precision * recall) / (precision + recall)
print("f1_score: ", f1_score)
# tính precision, recall. F1_score